In [13]:
import datetime
from pathlib import Path
import atomic as atom
import definitions as defs
import visuals as vis 
import pandas as pd

from ta.volatility import BollingerBands
from ta.trend import MACD
from ta.momentum import RSIIndicator
from ta.trend import IchimokuIndicator
from ta.trend import EMAIndicator
from ta.trend import SMAIndicator
from ta.trend import ADXIndicator
from ta.volatility import KeltnerChannel

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [44]:
def getOPData(date, EnterTime):
    Banknifty_Path = "../NIFTYOptionsData/OptionsData/Banknifty/"
    Nifty_Path = "../NIFTYOptionsData/OptionsData/Nifty/"
    date_string = date.strftime("%Y/Data%Y%m%d.csv")
    BNPath = Banknifty_Path + date_string
    NPath = Nifty_Path + date_string
    my_fileN = Path(NPath)
    my_fileBN = Path(BNPath)
    # print("Working on file - "+date_string)
    if my_fileN.exists() and my_fileBN.exists():
      masterdfN = atom.LoadDF(NPath)
      masterdfBN = atom.LoadDF(BNPath)
    masterdf = masterdfBN

    symbol = defs.BN
    spotdata = atom.GetSpotData(masterdf, defs.BN)
    for s in range(len(spotdata)):
        currentcandle = spotdata.iloc[s]
        if currentcandle.name.time() == EnterTime:
            exp = atom.GetExpiry(masterdf, symbol)
            if symbol == defs.N :
                cst = currentcandle["open"]
                cst = int(round(cst / 50, 0)*50)
            elif symbol == defs.BN :
                cst = currentcandle["open"]
                cst = int(round(cst / 100, 0) * 100)
            opdfCE = masterdf[masterdf['symbol'] == symbol + exp + str(cst) + "CE"]
            opdfPE = masterdf[masterdf['symbol'] == symbol + exp + str(cst) + "PE"]
            break
    return (opdfCE, opdfPE)


In [45]:
# Conditions with RSI
def LongEnterCondition(s, opdata, longRSIThresh, ADXThresh):
    if (opdata["rsi"][s] < longRSIThresh) and (opdata["rsi"][s] > opdata["rsi"][s-1]) and (opdata["diplus"][s] > opdata["diminus"][s]):
        return True
    else:
        return False
    
def ShortEnterCondition(s, opdata, shortRSIThresh, ADXThresh):
    if (opdata["rsi"][s] > shortRSIThresh) and (opdata["rsi"][s] < opdata["rsi"][s-1]) and (opdata["diminus"][s] > opdata["diplus"][s]):# and (stock["adx"][s] > ADXThresh):
        return True
    else:
        return False
    
def LongExitCondition(s, opdata, longposition, longRSIThresh, SLPc):
   # Exit Condition based on longexit
    if (opdata["rsi"][s] > longRSIThresh) or (opdata["diplus"][s] < opdata["diminus"][s]):
        return True
    elif (opdata["close"][s] - longposition["Buy Price"]) < -1*SLPc*opdata["close"][s]:
        return True
    else:
        return False

def ShortExitCondition(s, opdata, shortposition, shortRSIThresh, SLPc):
   # Exit Condition based on longexit
    if (opdata["rsi"][s] < shortRSIThresh) or (opdata["diminus"][s] < opdata["diplus"][s]):
        return True
    elif (shortposition["Sell Price"] - opdata["close"][s]) < -1*SLPc*opdata["close"][s]:
        return True
    else:
        return False


In [49]:
# Conditions only with DI
def LongEnterCondition(s, opdata, longRSIThresh, ADXThresh):
    if (opdata["diplus"][s] > opdata["diminus"][s]):
        return True
    else:
        return False
    
def ShortEnterCondition(s, opdata, shortRSIThresh, ADXThresh):
    if (opdata["diminus"][s] > opdata["diplus"][s]):# and (stock["adx"][s] > ADXThresh):
        return True
    else:
        return False
    
def LongExitCondition(s, opdata, longposition, longRSIThresh, SLPc):
   # Exit Condition based on longexit
    if (opdata["diplus"][s] < opdata["diminus"][s]):
        return True
    elif (opdata["close"][s] - longposition["Buy Price"]) < -1*SLPc*opdata["close"][s]:
        return True
    else:
        return False

def ShortExitCondition(s, opdata, shortposition, shortRSIThresh, SLPc):
   # Exit Condition based on longexit
    if (opdata["diminus"][s] < opdata["diplus"][s]):
        return True
    elif (shortposition["Sell Price"] - opdata["close"][s]) < -1*SLPc*opdata["close"][s]:
        return True
    else:
        return False


In [50]:
longRSIThresh = 70
ADXThresh = 30
shortRSIThresh = 40
longpositions = []
longposition = {}
shortpositions = []
shortposition = {}
SLPc = 0.1
win = 20
lotsize = 25

date = datetime.date(2022, 9, 23)
EnterTime = datetime.time(9,16)
(opdfCE, opdfPE) = getOPData(date, EnterTime)
count = 0
LongActive = False
ShortActive = False
TotalPNL = 0

Rsi = RSIIndicator(opdfCE['close'], window=win)
opdfCE["rsi"] = Rsi.rsi()
Adx = ADXIndicator(opdfCE['high'], opdfCE['low'], opdfCE['close'], window = win)
opdfCE["adx"] = Adx.adx()
opdfCE["diplus"] = Adx.adx_pos()
opdfCE["diminus"] = Adx.adx_neg()
opdfCE = opdfCE.dropna()

for s in range(len(opdfCE)):
    count = count + 1
    if (count < len(opdfCE)) and (s > 0):
        if (LongEnterCondition(s, opdfCE, longRSIThresh, ADXThresh)) and not LongActive:
            print("Buying Long")
            longposition["Buy Time"] = opdfCE["time"][s+1]
            longposition["Buy Price"] = opdfCE["open"][s+1]
            longposition["Date"] = opdfCE["date"][s+1]
            LongActive = True
        if LongActive and LongExitCondition(s, opdfCE, longposition, longRSIThresh, SLPc):
            print("Selling Long")
            longposition["Sell Time"] = opdfCE["time"][s+1]
            longposition["Sell Price"] = opdfCE["open"][s+1]
            longposition["PNL"] = longposition["Sell Price"] - longposition["Buy Price"]
            TotalPNL = TotalPNL + longposition["PNL"]
            longpositions.append(longposition)
            LongActive = False
            longposition = {}
longpos = pd.DataFrame(longpositions)
print(longpos)

print("Total PNL in " + str(TotalPNL*lotsize))

Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
    Buy Time  Buy Price        Date Sell Time  Sell Price    PNL
0   10:51:00     252.90  2022-09-23  10:58:00      253.25   0.35
1   10:59:00     257.75  2022-09-23  11:00:00      251.00  -6.75
2   11:25:00     245.70  2022-09-23  11:31:00      228.60 -17.10
3   11:50:00     240.95  2022-09-23  11:51:00      237.95  -3.00
4   11:54:00     242.95  2022-09-23  11:56:00      244.65   1.70
5   11:57:00     245.30  2022-09-23  11:58:00      234.25 -11.05
6   12:46:00     214.75  2022-09-23  12:47:00      211.05  -3.70
7   12:49:00     214.90  2022-09-23  12:51:00      209.45  -5.45
8   13:18:00     214.20  2022-09-23  13:27:00      204.05 -10.15
9   13:34:00     212.40  2022-09-23  13:37:00     

/Users/rishabhiyer/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/Users/rishabhiyer/opt/anaconda3/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/rishabhiyer/opt/anaconda3/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


In [51]:
longRSIThresh = 70
ADXThresh = 30
shortRSIThresh = 40
longpositions = []
longposition = {}
shortpositions = []
shortposition = {}
SLPc = 0.1
win = 20
lotsize = 25

date = datetime.date(2022, 9, 23)
EnterTime = datetime.time(9,16)
(opdfCE, opdfPE) = getOPData(date, EnterTime)
count = 0
LongActive = False
ShortActive = False
TotalPNL = 0

Rsi = RSIIndicator(opdfPE['close'], window=win)
opdfPE["rsi"] = Rsi.rsi()
Adx = ADXIndicator(opdfPE['high'], opdfPE['low'], opdfPE['close'], window = win)
opdfPE["adx"] = Adx.adx()
opdfPE["diplus"] = Adx.adx_pos()
opdfPE["diminus"] = Adx.adx_neg()
opdfPE = opdfPE.dropna()

for s in range(len(opdfPE)):
    count = count + 1
    if (count < len(opdfPE)) and (s > 0):
        if (LongEnterCondition(s, opdfPE, longRSIThresh, ADXThresh)) and not LongActive:
            print("Buying Long")
            longposition["Buy Time"] = opdfPE["time"][s+1]
            longposition["Buy Price"] = opdfPE["open"][s+1]
            LongActive = True
        if LongActive and LongExitCondition(s, opdfPE, longposition, longRSIThresh, SLPc):
            print("Selling Long")
            longposition["Sell Time"] = opdfPE["time"][s+1]
            longposition["Sell Price"] = opdfPE["open"][s+1]
            longposition["PNL"] = longposition["Sell Price"] - longposition["Buy Price"]
            TotalPNL = TotalPNL + longposition["PNL"]
            longpositions.append(longposition)
            LongActive = False
            longposition = {}
longpos = pd.DataFrame(longpositions)
shortpos = pd.DataFrame(shortpositions)
print(longpos)
print(shortpos)

print("Total PNL in " + str(TotalPNL*lotsize))

Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
Selling Long
Buying Long
    Buy Time  Buy Price Sell Time  Sell Price     PNL
0   09:37:00     708.45  10:37:00      845.00  136.55
1   10:45:00     915.75  10:51:00      860.70  -55.05
2   11:01:00     861.15  11:02:00      850.95  -10.20
3   11:05:00     868.00  11:13:00      856.40  -11.60
4   11:15:00     900.00  11:24:00      871.15  -28.85
5   11:31:00     903.30  11:47:00      859.40  -43.90
6   11:52:00     892.60  11:54:00      856.95  -35.65
7   11:59:00     871.70  12:00:00      867.95   -3.75
8   12:02:00     887.60  12:42:00     1005.00  117.4

/Users/rishabhiyer/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/Users/rishabhiyer/opt/anaconda3/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/rishabhiyer/opt/anaconda3/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
